In [27]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
import contractions
import unicodedata
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [28]:
spi = pd.read_csv('train_nourl.csv')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
# generating word frequencies
def gen_freq(text):
    # list of words
    word_list = []
    
    # loop over all the text docs and extract words into word_list
    for words in text.split():
        word_list.extend(words)
        
    # create word freuencies using word_list
    word_freq = pd.Series(word_list).value_counts()
    
    return word_freq

####################################################################
word_freq_normal = gen_freq(spi[spi['predator'] == 0]['msg'].str)
word_freq_predator = gen_freq(spi[spi['predator'] == 1]['msg'].str)
stop_words = stopwords.words('english')


#########################################
# removal of stop words and other symbols
smart_stopwords = stop_words 
ps = PorterStemmer()
pattern = r"[$&+,:;=_?@#|\[\]{}'<>.^*()%!-]"
def clean_text(text):
    text = ' '.join(re.sub(pattern, '', text).strip().split()) # remove punctuations
    text = ' '.join([word for word in text.split() if word.isalpha()]) # only words
    #text = BeautifulSoup(text).get_text() #html
    text = text.lower() #lower case
    text = contractions.fix(text) #contractions
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore') #symbols
    text = ' '.join(word for word in text.split() if word not in smart_stopwords) # stopwords
    text = ' '.join([ps.stem(word) for word in text.split()]) #stem
    return text
#########################################

#spi['msg'] = spi['msg'].apply(lambda x: clean_text(x))
#spi_sub = spi[spi['msg'].apply(lambda x: len(x.split()) > 1)]
#spi_sub.index = range(0, spi_sub.shape[0])
spi_sub = spi
spi_sub.head()

,Unnamed: 0,conversation id,author id,msg,time,predator
0,0.0,e621da5de598c9321a1d505ea95e6a2d,97964e7a9e8eb9cf78f2e4d7b2ff34c7,Hola.,0.138889,0.0
1,1.0,e621da5de598c9321a1d505ea95e6a2d,0158d0d6781fc4d493f243d4caa49747,hi.,0.138889,0.0
2,2.0,e621da5de598c9321a1d505ea95e6a2d,0158d0d6781fc4d493f243d4caa49747,whats up?,0.138889,0.0
3,3.0,e621da5de598c9321a1d505ea95e6a2d,97964e7a9e8eb9cf78f2e4d7b2ff34c7,not a ton.,0.138889,0.0
4,4.0,e621da5de598c9321a1d505ea95e6a2d,97964e7a9e8eb9cf78f2e4d7b2ff34c7,you?,0.138889,0.0


In [30]:
# splitting test train data
spi_sub.loc[(pd.isna(spi_sub["predator"])),['predator']] = 0
spi_sub.loc[(pd.isna(spi_sub["msg"])),['msg']] = "whatsup sexy"


# We need to make sure the classes have the same proportion in both sets
X_train, X_test, y_train, y_test = train_test_split(np.array(spi_sub['msg'].apply(lambda x: np.str_(x))), 
                                                    np.array(spi_sub['predator']), stratify = np.array(spi_sub['predator']),
                                                    test_size = 0.20, random_state = 100)

tv = TfidfVectorizer(min_df = 0., max_df = 1., norm = 'l2', use_idf = True, smooth_idf = True)

train_tfidf = tv.fit_transform(X_train)
test_tfidf = tv.transform(X_test)

print(train_tfidf.shape)
print(test_tfidf.shape)

(256467, 68459)
(64117, 68459)


In [31]:
log_model = LogisticRegression(penalty = 'l2', solver = 'lbfgs', multi_class = 'ovr', max_iter = 1000, C = 1, random_state = 100)
log_model.fit(train_tfidf, y_train)

log_predictions = log_model.predict(test_tfidf)
print(classification_report(y_test, log_predictions))


svm = LinearSVC(penalty = 'l2', C = 1, random_state = 100)
svm.fit(train_tfidf, y_train)

svm_predictions = svm.predict(test_tfidf)
print(classification_report(y_test, svm_predictions))

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98     61132
         1.0       0.59      0.04      0.08      2985

    accuracy                           0.95     64117
   macro avg       0.77      0.52      0.53     64117
weighted avg       0.94      0.95      0.93     64117

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98     61132
         1.0       0.54      0.05      0.10      2985

    accuracy                           0.95     64117
   macro avg       0.75      0.53      0.54     64117
weighted avg       0.94      0.95      0.94     64117



In [39]:

sm = SMOTE(random_state =2)
X_sm, y_sm = sm.fit_sample(train_tfidf, y_train)

log_model = LogisticRegression(penalty = 'l2', solver = 'lbfgs', multi_class = 'ovr', max_iter = 1000, C = 1, random_state = 100)
log_model.fit(X_sm, y_sm)

log_sm_predictions = log_model.predict(test_tfidf)
print(classification_report(y_test, log_sm_predictions))



svm = LinearSVC(penalty = 'l2', C = 1, random_state = 100)
svm.fit(X_sm, y_sm)

svm_sm_predictions = svm.predict(test_tfidf)
print(classification_report(y_test, svm_sm_predictions))

              precision    recall  f1-score   support

         0.0       0.99      0.73      0.84     61132
         1.0       0.13      0.80      0.22      2985

    accuracy                           0.73     64117
   macro avg       0.56      0.76      0.53     64117
weighted avg       0.95      0.73      0.81     64117

              precision    recall  f1-score   support

         0.0       0.99      0.73      0.84     61132
         1.0       0.12      0.80      0.22      2985

    accuracy                           0.73     64117
   macro avg       0.56      0.76      0.53     64117
weighted avg       0.95      0.73      0.81     64117



In [ ]:
#Training 
from sklearn import svm

X_train = X_sm
y_train = y_sm

Lr=LogisticRegression(penalty = 'l2', solver = 'lbfgs', multi_class = 'ovr', max_iter = 1000, C = 1, random_state = 100)
LRRR=LinearRegression()
SVM = svm.LinearSVC()
RF = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
NN = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

#print("Logistic Regression")
#Lr.fit(X_train,y_train)

#print("Linear Regression")
#LRRR.fit(X_train,y_train)

#print("SVM Regression")
#SVM.fit(X_train, y_train)

#print("RF Regression")
#RF.fit(X_train, y_train)

#print("NN Regression")
NN.fit(X_train, y_train)


#print(Lr.score(X_train,y_train))
#print(LRRR.score(X_train,y_train))
#print(SVM.score(X_train,y_train))
#print(RF.score(X_train,y_train))
print(NN.score(X_train,y_train))


In [34]:
X_test = test_tfidf

#print(log_model.score(X_test,y_test))
#print(LRRR.score(X_test,y_test))
#print(SVM.score(X_test,y_test))
#print(RF.score(X_test,y_test))
print(NN.score(X_test,y_test))

0.9540683438089742


In [37]:

msg = "bro we will go put on trip fast"
msg = msg.split()
msg = np.array(msg)
msg = tv.transform(msg)
print(log_model.predict(msg))

[0.]


In [64]:
X_train, X_test, y_train, y_test = train_test_split(np.array(spi_sub['msg'].apply(lambda x: np.str_(x))), 
                                                    np.array(spi_sub['predator']), stratify = np.array(spi_sub['predator']),
                                                    test_size = 0.20, random_state = 100)
spi_sub[spi_sub['predator'] == 1]["msg"]
for i in X_test:
    print(i)

i hav 34 C chest
whatsup sexy
is there any way to avoid that?
im not
hi, ne1 know why azureus will download normally for a while then it stops downloading and still uploads?
Living XHR
or a vid
;)
http://jsfiddle.net/mk7sg/2/ works just fine
i just set them for write-through and deal with it
b25b6b77a0087ff8385941e5545d32ea: 0-based indeces are natural to computer because of pointer arithmetic, but some algorithms for computing positions in array work only with 1-based indeces because of various properties of number one
like how
hey
the scientific method is the main thing i got from my degree which i've applied to the spec work
painting
yea
I suppose
you're missing what I'm suggesting.. as a way to identify specifically the css version that you're coding for, if future changes come out, the site would still display properly with the css version it's specified for.
so depending on what you mean with &quot;Method-Stuff&quot;...
TRUE
but I do think JSONRequest is not a great security desi

Hii
and UTF-9/UTF-18 were implemented in PDP-10 assembly language apparently
Wazzupp
hi
227a9a17d1db3f8e5c71a150345fb2a6:  happy to prettify / unify these
unfortunately the uml creation in NB is in the enterprise pack :/
so is possible tu use srvmgr and usrmgr to manage samba??????
I've been able to remake some classic games in canvas using a common Game class, I'm just wondering if there's any interest out there for me to release it
so im 69
mmm nvm are u a virgin
honestly, with my personal traffic, that'd be a lot of people, but if they get 1000s daily or so, 50 isn't much
just realized that :)
xD
kk
b25b6b77a0087ff8385941e5545d32ea, I'm sure some stuff will break, we'll have to figure out how much we can battle it with alpha builds
soon to be *much* nicer when everything works with bootable zfs
tell me what you want nexr
r u a boy?
male
what kinda music u like??
ME 2
it&apos;s sunday and I have tons of paper to do
i'll fix it
How are you?
i've not experienced this kind of rush befor

how do you like it there?
hey where you from?
'Florida Pharmacy Benefits Program' is the URL?
Np
aa99a6e1af27ad37111fe1bad3e0094f, ok, loaded
Kylratix and you do this why? you know even all Micro$oft office employee pc's run as admin
az se chudq drugo...dali da namerq lipswashtiq index i ot nego da mu ukazwam che wseki sledwasht e -1
Gooood morning 2 all
Hmm, guess again.
so how many times have you kissed a guy?
there are no australian sites that offer internet feeds of the ollympics?
ok, will do
69b0d3dfe919a6b860a9fac82de52a7e, there is no &quot;diff given below&quot; at http://www.w3.org/Bugs/Public/show_bug.cgi?id=9828
a190a2299f464789ce3270850f4ac49d: Hehe, unfortunately, mine isn't either. Most of the POIs are more or less on the same altitude, though
most patchsets are about the size of a cd...
are you male?
I forget
so
hi
PUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDI
    PUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUDDIPUD

Yes that will be fine :*
so while I declare the classes in .h files and put the code in .cpp files Ishould have no problem, right?
nice
yeah, I know, but I wanted to check what's in the w3c's spec
How would one best link to social profiles in a hCard?
megaspaz: i don't get it.
asian ^-^ and curvy
hahaha
?
..... okay
And is he in CSS-land or DOM-land?
as opposed to $500 with Apple
like know love you still
or extranets
68ad46d16b05645111d6dcb1b0cddc41: ‚Ä¶ transposed, but still set matrix3d the same way like you
do you have facebook account ?
Ok Portable Object Relocation Management.  Nice hidden joke if we can fit is somewhere.
or its inactive ?
most of the things of this type that the Chairs post get no responses at all
wanna try them? ;)
whats ur name
Hey
[http://twitter.com/kevinmarks] #tpac #semsyn @t the right thing to do is develop an open vocabulary first, then worry about the syntactic mapping to #microformats et al
how to replace every match with a value returned by the functio

the full city name is &quot;Krung Thep Maha Nakhon Amon Rattanakosin Mahinthara Ayutthaya Mahadilok Phop Noppharat Ratchathani Burirom Udom Ratchaniwet Mahasathan Amon Phiman Awatan Sathit Sakkathattiya Witsanu Kamprasit&quot;
What about you ?
^^
14, m, dk ;)
&lt;input&gt; is next
not too bad coulda been better
horny
hello
regrets+ Steve Faulkner
u r nice
me 2!!!
apt-cache search hibernate
14101c52afe433b7510c49db36d2e500: I don't like the fact that it makes my life an absolute hell with not supporting generated markup
itch =&gt; scratch =&gt; release :)
yw
lol
asl?
f or m
....
hows it goin
!winerror 21
>:D<:-*
V
69b0d3dfe919a6b860a9fac82de52a7e, I think <email/> would be a good start (cc me please)
does having no media resource count as fail though?
Sh4d03, listamatic: <a href="http://css.maxdesign.com.au/listamatic/">http://css.maxdesign.com.au/listamatic/</a>
Well, I love the taste of a woman&apos;s body... I love the feeling of lips and a tongue on me...
just wondering
you
yeah... 

...

she played soccer
we are not listening anyway :-)
yeah phrakture
(Well, impossible for a generic algorithm to serialise)
Hey
Blah. I went out of my way to make sure my AJAX apps at least are accessible in Konq. Some of my event handlers don't work but the page functions.
Hi!
euhm.. okay, so I'm starting with a basic application but I'm still having problems...
otherwise safe
J4k3: sr9
done, e-mailed update to public-css-testsuite and checked in new and revised tests
what state?
finee
but the first who didn&apos;t to cyber
AYYYEEE
i AM nice
yea
or something
i know haven't had my sexy gurl around
:-)
i'm trying to use &quot;allow from &lt;ip&gt;&quot; for a directory, but it doesn't work.  only &quot;allow from &lt;hostname&gt;&quot; works.  any idea why?
a924fb58d2c25874322f4e5126f608b9: I like how the subtitles get completely obscured by the popup advert
no
it seems IE8b1 doesn't, though
should &quot;&lt;![CDATA[&quot; be case-insensitive?
hi
bbl
no fire is also used for events with ot

entry-15, drinks-50-taxis30
i want my query to return &quot;2&quot;
awdawdawd
mmm
hi
no i wna hve sex wd u
ok
nothing much happened last week but maybe I misremember?
that sure would be nice wouldn't it
oh, it's actually git aware
he goes in n comes home at diffrent times cuz hes a doctor
so &lt;img class= photo&gt; would be invalid, but &lt;img class= &quot;photo&quot; could still be valid
If the charter changes, would it change so that it more explicitly includes this graphic stuff, or might it change to explicitly exclude it instead, or might it change and not clarify anything at all?
your lesbain?
what ended up being the actual problem, or do you not know?
edb259c0e0038f38bb200bc20c8cbf7e: i thought i'd get something automatically by setting ie to default browser and doing a windows update
hey
damn i am getting close to cum
asl
hi
i dont remember well
those are very different
yep
red thong?
???????
ya
pano ?
oh
http://boagworld.com/forum/comments.php?DiscussionID=2400
no
asl?
wb ma

good good
oh ok
how old r u
ok
you?
I&apos;m spent, I jump down into the alley and smoke a fatty. I throw rocks at the cats.
I've never had trouble with the no-ip updater
Nope.. :(
ok you have a meter?
Blending rgba(0,255,0,0.5) on top of rgba(0,255,0,1) gives dark green, in APNG with default gamma, which is annoying because it means I'll have to learn how gamma works
hi
The spec could state that, due to grammar overrides in the prose, the grammar is not sufficient by itself for creating a parser
No
hiiiiiiiiiiii
ok
why does my netrw tab closes randomly when I re-open it? O_o
-_-
hi sweety
it says your running a different version of yahoo
8b756423dbbfd4f4ebb546916a327a65: Not like being supported renders it much different anyway.
lol
whatsup sexy
yeah, i think so
I think I should make more detailed instructions :P.
oeh geil
as
age
or changing the ASCII-compatibility requirements to exclude even ascii bytes in later parts of multibyte sequences
do i know? what?
:D :D
http://www.teenporn

im 21
http://wearehugh.com/public/2010/08/html5-video-accessibility/ heh
me 2
hi
how can awnser without give me the all the work already done?? lol :)
from?
that's my point
ntr qt chat!!
Opera too.
lies
damieng-work: to me they're the same :)
how do i know your not 17, just fucking with me! :P
not that i really expected different
Anne is one of those 3
[[Special:Log/block]] block  * Tantek *  blocked [[User:DaisiBadger880]] with an expiry time of infinite (account creation disabled): Spamming links to external sites
any f?
yeah, ill be better after a while
?
Hello World
ur asl plz
Ho
hey
Nope
U don't care I'm smoke?
you should be binding to whatever the manager cn you have setup in slapd.conf is
asl?
hi asl
hey
hello
right, I'm saying it should probably disallow it
18
+probably. I know it's covered in:
that sucks
I&apos;m female.
hey
BCMM: if done badly it is about the same as the paste. Done well, it is 3-4C cooler.
ab7faae5d7b250ea8606486575f8f79c, ty that was my next question lol
i'

you can map whatever you want to that
i have a chrooted apache
I was just wondering about the peculiar choice of nickname
awful!
asl?
I suspect that the dB value isn't that reliable...
defining side effects that can happen during GC is nasty, because GC is very non-deterministic
boston is very in neighbor of new york, yes?
lol
listen bitch you dont ask questions
but it seems like i'd have to replace ram as well
I don't think that has really been the problem with IANA, it's just that the registries lacked everything else that was out there
hi
21 f aus
why/
brianleroux++
SMIL isn't really intended for &quot;eye candy&quot;: it's a content language.
hello
Visit Http://www.tuxhosting.net NOW ! , For Cheap Hosting ,Which has an effective and Relaible Service
whos suzi creamcheese
asl
only 1 HD is selected as the boot device by the BIOS
oh. Is she beutiful?
i just need to prompt the user &quot;stop these programs first&quot;
and i was supposed to join him on his fucking mission
hey!!!
asl?


In [38]:
import pickle


pkl_filename = "Models/Lr_model_1.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(Lr, file)
    



In [17]:
import pickle
loaded_model = pickle.load(open("Models/Lr_model_1.pkl", 'rb'))




In [33]:
msg = '#love#instagood#photooftheday#fashion#Beautiful#like4like#picoftheday#art#happy#photography#instagram#followme#style#follow#instadaily#travel#life#cute#fitness#nature#beauty#girl#fun#photo#amazing#likeforlike#instalike#Selfie#smile#me#lifestyle#model#follow4follow#music#friends#motivation#like#food#inspiration#Repost#summer#design#makeup#TBT#followforfollow#ootd#Family#l4l#cool#igers#TagsForLikes#hair#instamood#sun#vsco#fit#beach#photographer#gym#artist#girls#vscocam#autumn#pretty#luxury#instapic#black#sunset#funny#sky#blogger#hot#healthy#work#bestoftheday#workout#f4f#nofilter#london#goals#blackandwhite#blue#swag#health#party#night#landscape#nyc#happiness#pink#lol#foodporn#newyork#fitfam#awesome#fashionblogger#Halloween#Home#fall#paris'
msg = msg.split()
msg = np.array(msg)
msg = tv.transform(msg)
print(log_model.predict(msg))

[0.]
